In [14]:
import numpy as np
import pandas as pd



train_0_path = "/kaggle/input/ps04-tech-gc-iitd/train_0.parquet"
df_0 = pd.read_parquet(train_0_path) 
df_0.head(10)
print(len(df_0))

972105


In [15]:
def get_train_path(id):
    path = f"/kaggle/input/ps04-tech-gc-iitd/train_{id}.parquet"
    return path
#random dfs
to_use = [0,1,4,5]
dfs = [] 
for i in to_use:
    df = pd.read_parquet(get_train_path(i))
    dfs.append(df)
    print("added df of size", len(df))
    
df = pd.concat(dfs, ignore_index=True)

print("size of combiend df i am using is ", len(df))
df = df.sort_values(by=['date_id', 'time_id'])

added df of size 972105
added df of size 972105
added df of size 1518436
added df of size 1518437
size of combiend df i am using is  4981083


In [16]:
nan_counts = df.isna().sum()
for cnt in nan_counts:
    print(cnt, end=', ')

0, 0, 0, 0, 1944210, 1944210, 1944210, 1944210, 1944210, 0, 0, 0, 16980, 0, 0, 0, 0, 0, 0, 140852, 86, 23568, 79, 79, 0, 4981083, 0, 0, 0, 0, 4981083, 4981083, 0, 0, 0, 4981083, 55489, 55489, 0, 0, 0, 0, 0, 785673, 45674, 200367, 785673, 45674, 200367, 195722, 195722, 87, 0, 0, 750976, 5867, 164276, 750976, 5867, 164276, 79, 79, 55484, 0, 0, 0, 184069, 151988, 156957, 195722, 195722, 0, 0, 0, 0, 0, 0, 55484, 55484, 208, 208, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [17]:
# dropping all rows with nan not desirable since too many of them (~75 %) 
# so impute them, setting all to 0 for now
def impute(to_impute):
    df_filled = to_impute.fillna(0)  # Replace NaNs with 0 :)
    # df_filled = to_impute.fillna(method='ffill')  # Forward fill (uses previous value)
    # df_filled = df_filled.fillna(0) # fill remaining with 0
    return df_filled

impute_df = impute(df) 


In [69]:
# splitting by symbol
df_sorted = impute_df.sort_values(['symbol_id', 'date_id', 'time_id'])
symbol_dfs = {symbol: df_sorted[df_sorted['symbol_id'] == symbol].copy() 
              for symbol in df_sorted['symbol_id'].unique()}

In [19]:
# baseline: lin reg with weights
ids = ["00", "01", "02", "03", "04", "05", "06", "07", "08", "09"]
ids.extend([str(i) for i in range(10,79)])
features_to_use = [f"feature_{id}" for id in ids] 
features_to_use.extend(['date_id', 'time_id']) 

In [83]:
from sklearn.linear_model import LinearRegression

def train_lin_reg_model(df_train, df_test, features_to_use, target_var, weights):
    X = df_train[features_to_use]
    Y = df_train[target_var]
    print("X:" , len(X))
    print("Y: ", len(Y))
    sample_weights = df_train[weights]
    model = LinearRegression()
    print("fitting lin reg model...")
    model.fit(X, Y, sample_weight=sample_weights)

    
    X_test = df_test[features_to_use]
    Y_test = df_test[target_var] 
    sample_weights_test = df_test[weights]
    r2 = model.score(X_test, Y_test, sample_weight=sample_weights_test) 
    print("r2 on test data is ", r2) 
    return r2, model
    
    


In [84]:
# create train test split
# currently working on symbol 0

def train_lin_reg_from_df(curr_df, features):
    train_size = int(0.8 * len(curr_df))  # 80% train, 20% test
    curr_df = curr_df.dropna()
    # Split the data
    train = curr_df.iloc[:train_size]
    test = curr_df.iloc[train_size:]
    print("train size is ", len(train))
    print("test size is ", len(test))
    for c in features:
        if c not in train.columns:
            print("c :", c, " missing in train")
    
    r2, model = train_lin_reg_model(train, test, features, "target_1", "weight")
    return r2, model

# symbols = symbol_dfs.keys()
# r2_list = []
# for i in symbols:
#     r2 = train_lin_reg_from_df(symbol_dfs[i], features_to_use)
#     r2_list.append(r2)

In [22]:
# aim to use lag level features 

def add_lag_feature(lag, f, features, df):
    for j in range(1, lag + 1):
            df[f"{f}_{j}"] = df[f].shift(j) 
    
    df[f"{f}_{lag}_mean"] = df[f"{f}"].rolling(window=lag+1).mean()
    df[f"{f}_{lag}_std"] =  df[f"{f}"].rolling(window=lag+1).std()


In [23]:
working_df = symbol_dfs[0]

In [24]:
to_ignore = ["time_id", "date_id"]
lag = 3
lag_level_features = [i for i in features_to_use]
for i in features_to_use:
    if i in to_ignore:
        continue
    add_lag_feature(lag, i, features_to_use, working_df)
    lag_level_features.append(f"{i}_{lag}_mean")
    lag_level_features.append(f"{i}_{lag}_std")
    for j in range(1,lag + 1):
        lag_level_features.append(f"{i}_{j}")



for j in range(2, lag + 1):
    lag_level_features.append(f"target_1_{j}")

lag_level_features.append(f"target_1_{lag}_mean")
lag_level_features.append(f"target_1_{lag}_std")


<ipython-input-22-792c824ae36c>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{f}_{j}"] = df[f].shift(j)
<ipython-input-22-792c824ae36c>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{f}_{lag}_mean"] = df[f"{f}"].rolling(window=lag+1).mean()
<ipython-input-22-792c824ae36c>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented f

In [25]:
# working_df = working_df.drop(columns = ["target_1_1"]) 
cols = working_df.columns
for col in lag_level_features:
    if ("target_1" in col):
        print(col)
row = working_df.iloc[10]
print(row["target_1"])
# print(row["target_1_1"])

target_1_2
target_1_3
target_1_3_mean
target_1_3_std
4.31430721282959


In [28]:
r2_dict = dict()
for symbol,_ in symbol_dfs.items():
    working_df = symbol_dfs[symbol]
    for i in features_to_use:
        if i in to_ignore:
            continue
        add_lag_feature(lag, i, features_to_use, working_df)

    for j in range(2, lag + 1):
        working_df[f"target_1_{j}"] = working_df["target_1"].shift(j)
    working_df[f"target_1_{lag}_mean"] = working_df[f"target_1"].rolling(window=lag+1).mean()
    working_df[f"target_1_{lag}_std"] =  working_df[f"target_1"].rolling(window=lag+1).std()

    
    r2 = train_lin_reg_from_df(working_df, lag_level_features)
    r2_dict[symbol] = r2

train size is  214627
test size is  53654
X: 214627
Y:  214627
fitting lin reg model...
r2 on test data is  0.952371224459525
train size is  230248
test size is  57560
X: 230248
Y:  230248
fitting lin reg model...
r2 on test data is  0.9571215600720834


<ipython-input-22-792c824ae36c>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{f}_{j}"] = df[f].shift(j)
<ipython-input-22-792c824ae36c>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{f}_{lag}_mean"] = df[f"{f}"].rolling(window=lag+1).mean()
<ipython-input-22-792c824ae36c>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented f

train size is  173875
test size is  43466
X: 173875
Y:  173875
fitting lin reg model...
r2 on test data is  0.9654637237363426


<ipython-input-22-792c824ae36c>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{f}_{j}"] = df[f].shift(j)
<ipython-input-22-792c824ae36c>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{f}_{lag}_mean"] = df[f"{f}"].rolling(window=lag+1).mean()
<ipython-input-22-792c824ae36c>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented f

train size is  162328
test size is  40580
X: 162328
Y:  162328
fitting lin reg model...
r2 on test data is  0.9652000917910835


...

train size is  164366
test size is  41089
X: 164366
Y:  164366
fitting lin reg model...
r2 on test data is  0.9628468764462008


<ipython-input-22-792c824ae36c>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{f}_{j}"] = df[f].shift(j)
<ipython-input-22-792c824ae36c>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{f}_{lag}_mean"] = df[f"{f}"].rolling(window=lag+1).mean()
<ipython-input-22-792c824ae36c>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented f

train size is  17659
test size is  4412
X: 17659
Y:  17659
fitting lin reg model...
r2 on test data is  0.9675252906482696


<ipython-input-22-792c824ae36c>:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{f}_{j}"] = df[f].shift(j)
<ipython-input-22-792c824ae36c>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{f}_{lag}_mean"] = df[f"{f}"].rolling(window=lag+1).mean()
<ipython-input-22-792c824ae36c>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented f

train size is  222098
test size is  55522
X: 222098
Y:  222098
fitting lin reg model...
r2 on test data is  0.9666567394924371


In [ ]:
# row = working_df.iloc[10]
# print(row)
# lag_level_features.append("target_1_1")
# lag_level_features = [x for x in lag_level_features if x != "target_1_1"]
train_lin_reg_from_df(working_df, lag_level_features)
# train_lin_reg_from_df(symbol_dfs[0], features_to_use)


In [29]:
print(r2_dict)

{0: 0.952371224459525, 1: 0.9571215600720834, 2: 0.9654637237363426, 3: 0.9652000917910835, 5: 0.9619943359473908, 7: 0.9619642715194875, 8: 0.9630853367932974, 9: 0.9595013551209803, 10: 0.9631120292629866, 11: 0.9688614013023816, 12: 0.9665639377284585, 13: 0.9672104557883899, 14: 0.9666595128387843, 15: 0.9651295309659796, 16: 0.961825375941538, 17: 0.9646316736817825, 19: 0.9646897428594722, 20: 0.9603644173778577, 21: 0.9474858867395762, 22: 0.9619242011927285, 23: 0.9440435095803849, 25: 0.9665482827283104, 26: 0.9667952168689633, 29: 0.9664427232468332, 30: 0.9652868446097476, 33: 0.9587222570740411, 34: 0.9628468764462008, 36: 0.9675252906482696, 38: 0.9666567394924371}


In [91]:
def weighted_r2(y_true, y_pred, weights):
    # y_true =  y_true.to_numpy()
    # y_pred = y_pred.to_numpy()
    # weights = weights.to_numpy()
    print(y_pred[0], y_pred[1])
    # y_mean = np.average(y_true, weights=weights)  # Compute weighted mean of y
    numerator = np.sum(weights * (y_true - y_pred) ** 2)  # Weighted squared error
    denominator = np.sum(weights * (y_true - y_true.mean()) ** 2)  # Weighted variance
    return 1 - (numerator / denominator)

In [61]:
target_features = ["target_1_3_mean"]
train_lin_reg_from_df(working_df, target_features) 

train size is  214627
test size is  53654
X: 214627
Y:  214627
fitting lin reg model...
r2 on test data is  0.8414173736229538


0.8414173736229538

In [93]:
working_df = symbol_dfs[0]
working_df["target_1_1"] = working_df["target_1"].shift(1)
working_df["target_1_2"] = working_df["target_1"].shift(2)
working_df["target_1_3"] = working_df["target_1"].shift(3)
working_df["target_1_3_mean"] = (working_df["target_1_1"] + working_df["target_1_2"] + working_df["target_1_3"]) / 3.0 
target_features = ["target_1_1", "target_1_2", "target_1_3", "target_1_3_mean"]
r2, model = train_lin_reg_from_df(working_df, target_features)


train size is  214627
test size is  53654
X: 214627
Y:  214627
fitting lin reg model...
r2 on test data is  0.765236471918638


In [94]:
# verifying
train_split = int((0.8 * len(working_df)))
test_df = working_df[train_split:]
test_targets = test_df["target_1"]
test_weights = test_df["weight"]
test_df = test_df[["target_1_1", "target_1_2", "target_1s_3", "target_1_3_mean"]].copy()
test_pred = model.predict(test_df.to_numpy())


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [95]:
weighted_r2(test_targets.to_numpy(), test_pred,  test_weights.to_numpy())

-0.1799217 -0.25059944


0.7652191072702408